# Import data

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("grassknoted/asl-alphabet")

print("Path to dataset files:", path)

import os

DATA_DIR = os.path.join(
    path,
    "asl_alphabet_train",
    "asl_alphabet_train"
)


# Import lib

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


# Chuan hoa du lieu

In [ ]:
IMG_SIZE = 64
BATCH_SIZE = 128 # bn hinh 1 lan chay epoch

In [ ]:
# cai thien datagen
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)


In [ ]:
train_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training' # chi dinh lay tap train
)

val_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation' # chi dinh lay tap validation
)

# Khai bao mo hinh - fit

In [ ]:
# Add BatchNormalization and stronger regularization
from tensorflow.keras.layers import BatchNormalization

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # increase dropout
    Dense(29)  # no activation here, use from_logits=True
])

model.summary()

In [ ]:
# huan luyen
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
    metrics=['accuracy']
)
history = model.fit(
    train_data,
    epochs=10,
    validation_data=val_data
)

# Save model

In [ ]:
model.save('asl_alphabet_model.h5') # .h5: dung de load lai va du doan cho sau nay
print('Saved model')

# Danh gia mo hinh

In [ ]:
loss, accuracy = model.evaluate(val_data)
print(f'Accuracy: {accuracy:.4f}')
print(f'Loss: {loss:.4f}')

# Du doan

In [ ]:
# load mo hinh 
loaded_model = keras.models.load_model('asl_alphabet_model.h5')
print('Loaded model from disk')

In [ ]:
# chuan bi label (chu cai)
class_names = list(train_data.class_indices.keys())
print(class_names)

In [ ]:
# load 1 hinh de xu ly
img_dir = os.path.join(path, "asl_alphabet_train", "asl_alphabet_train", "M")  # duong dan den thu muc chua hinh
# Get first image file
img_file = os.listdir(img_dir)[0]
img_path = os.path.join(img_dir, img_file) # duong dan den hinh can du doan
# img_path = input("Enter the path of the image to predict: ")
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # scale anh
img_array = np.expand_dims(img_array, axis=0)  # them kich thuoc

In [ ]:
# du doan
prediction = loaded_model.predict(img_array)
predicted_index = np.argmax(prediction)
predicted_label = class_names[predicted_index] # map index -> label

# print % cac lop du doan
print(prediction)
print(f'Predicted label: {predicted_label}')